In [1]:
import sys
!{sys.executable} "-m pip install pandas requests tqdm"


'c:\Users\WINDOWS' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
!pip install pandas requests tqdm



Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: C:\Python313\python.exe -m pip install --upgrade pip


In [3]:
!{sys.executable} -m pip install pandas requests tqdm

'c:\Users\WINDOWS' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
import requests
import pandas as pd
from tqdm import tqdm

# --- CONFIGURATION ---
AUTH_ID = "54a4c77e-8649-1ccf-d4cf-cbd2fcbe0688"
AUTH_TOKEN = "mOVuWjU3mi3Gje7ft9yn"
SMARTY_URL = "https://us-street.api.smarty.com/street-address"

# --- LOAD INPUT ---
df = pd.read_csv("sample.csv")

# --- NEW OUTPUT COLUMNS ---
df["StandardizedAddress"] = ""
df["DPVStatus"] = ""
df["Latitude"] = ""
df["Longitude"] = ""
df["AddressStatus"] = ""

# --- PROCESS EACH ADDRESS ---
for i, row in tqdm(df.iterrows(), total=len(df)):
    street = f"{row['Number']} {row['Street']}"
    city = row.get("Suburb", "")
    state = row.get("State", "CA")
    zipcode = str(row.get("PostalCode", ""))

    params = {
        "auth-id": AUTH_ID,
        "auth-token": AUTH_TOKEN,
        "street": street,
        "city": city,
        "state": state,
        "zipcode": zipcode,
    }

    try:
        response = requests.get(SMARTY_URL, params=params, timeout=10)

        # Error 402 = no more lookups
        if response.status_code == 402:
            print(f"⚠️ Out of lookups for {street}. Check your API balance.")
            df.loc[i, "AddressStatus"] = "API limit reached"
            continue
        elif response.status_code != 200:
            print(f"⚠️ Error {response.status_code} for {street}")
            df.loc[i, "AddressStatus"] = f"Error {response.status_code}"
            continue

        data = response.json()
        if not data:
            df.loc[i, "AddressStatus"] = "No match"
            continue

        # Take the first matched result
        result = data[0]
        components = result.get("components", {})
        metadata = result.get("metadata", {})
        analysis = result.get("analysis", {})

        standardized = result.get("delivery_line_1", "")
        if "delivery_line_2" in result:
            standardized += " " + result["delivery_line_2"]
        standardized += ", " + result.get("last_line", "")

        # Fill in standardized data
        df.loc[i, "StandardizedAddress"] = standardized
        df.loc[i, "DPVStatus"] = analysis.get("dpv_match_code", "")
        df.loc[i, "Latitude"] = metadata.get("latitude", "")
        df.loc[i, "Longitude"] = metadata.get("longitude", "")
        df.loc[i, "AddressStatus"] = "Validated"

    except Exception as e:
        print(f"⚠️ Exception for {street}: {e}")
        df.loc[i, "AddressStatus"] = "Error"

# --- SAVE OUTPUT ---
df.to_csv("standardized_output.csv", index=False)
print("✅ Processing complete! Results saved to standardized_output.csv")


Using Smarty & LocIQ

In [1]:
import requests
import pandas as pd
from tqdm import tqdm
import time
from datetime import datetime
import re

# --- CONFIGURATION ---
AUTH_ID = "c158cf93-8a4d-c43c-dade-f372cbeac850"
AUTH_TOKEN = "e7UMwEf0GevqQpxwdLaO"
SMARTY_URL = "https://us-street.api.smarty.com/street-address"

# ⚙️ LocationIQ Fallback
LOCATIONIQ_URL = "https://us1.locationiq.com/v1/search"
LOCATIONIQ_KEY = "pk.f25c0d747a60b0c3c2fc8632d2064efa"  # 👈 replace with your own valid key

# --- LOAD INPUT ---
df = pd.read_csv("sample-input.csv")

# --- ADD OUTPUT COLUMNS ---
for col in ["StandardizedAddress", "DPVStatus", "Latitude", "Longitude", "AddressStatus"]:
    if col not in df.columns:
        df[col] = ""

# --- HELPER: LocationIQ fallback geocoder ---
def fallback_geocode(address):
    try:
        params = {"key": LOCATIONIQ_KEY, "q": address, "format": "json", "limit": 1}
        r = requests.get(LOCATIONIQ_URL, params=params, timeout=10)
        data = r.json()
        if isinstance(data, list) and len(data) > 0:
            lat = data[0].get("lat", "")
            lon = data[0].get("lon", "")
            display_name = data[0].get("display_name", "")
            return lat, lon, display_name
        else:
            return None, None, None
    except Exception as e:
        print(f"⚠️ LocationIQ error for {address}: {e}")
        return None, None, None

# --- MAIN PROCESSING LOOP ---
for i, row in tqdm(df.iterrows(), total=len(df)):
    street = f"{row.get('Number', '')} {row.get('Street', '')}".strip()
    city = row.get("Suburb", "")
    state = row.get("State", "CA")
    zipcode = str(row.get("PostalCode", "")).replace(".0", "").strip()

    # --- Call Smarty API ---
    params = {
        "auth-id": AUTH_ID,
        "auth-token": AUTH_TOKEN,
        "street": street,
        "city": city,
        "state": state,
        "zipcode": zipcode,
    }

    try:
        response = requests.get(SMARTY_URL, params=params, timeout=10)

        if response.status_code == 402:
            print(f"⚠️ Out of lookups for {street}. Using fallback only.")
            df.loc[i, "AddressStatus"] = "API limit reached"

        elif response.status_code != 200:
            print(f"⚠️ Error {response.status_code} for {street}")
            df.loc[i, "AddressStatus"] = f"Error {response.status_code}"

        else:
            data = response.json()
            if data:
                result = data[0]
                components = result.get("components", {})
                metadata = result.get("metadata", {})
                analysis = result.get("analysis", {})

                standardized = result.get("delivery_line_1", "")
                if "delivery_line_2" in result:
                    standardized += " " + result["delivery_line_2"]
                standardized += ", " + result.get("last_line", "")

                df.loc[i, "StandardizedAddress"] = standardized
                df.loc[i, "DPVStatus"] = analysis.get("dpv_match_code", "")
                df.loc[i, "Latitude"] = metadata.get("latitude", "")
                df.loc[i, "Longitude"] = metadata.get("longitude", "")
                df.loc[i, "AddressStatus"] = "Validated"
            else:
                df.loc[i, "AddressStatus"] = "No match"

    except Exception as e:
        print(f"⚠️ Exception for {street}: {e}")
        df.loc[i, "AddressStatus"] = "Error"

    # --- Fallback for missing coordinates ---
    lat_val = df.loc[i, "Latitude"]
    lon_val = df.loc[i, "Longitude"]

    if (not lat_val or str(lat_val) in ["", "0", "0.0", "nan", "None"]) or \
       (not lon_val or str(lon_val) in ["", "0", "0.0", "nan", "None"]):

        full_addr = f"{street}, {city}, {state} {zipcode}".strip()
        lat, lon, display_name = fallback_geocode(full_addr)

        if lat and lon:
            df.loc[i, "Latitude"] = lat
            df.loc[i, "Longitude"] = lon

            # 🧹 Clean and standardize parts (avoid NaN)
            number_part = str(row.get("Number", "")).strip() if pd.notna(row.get("Number")) else ""
            street_part = str(row.get("Street", "")).strip() if pd.notna(row.get("Street")) else ""
            city_part = str(row.get("Suburb", "")).strip() if pd.notna(row.get("Suburb")) else ""
            state_part = str(row.get("State", "")).strip() if pd.notna(row.get("State")) else "CA"
            zip_part = str(row.get("PostalCode", "")).strip() if pd.notna(row.get("PostalCode")) else ""

            # 📦 Normalize common street suffixes
            street_part = re.sub(r'\b(avenue|ave)\b', 'Ave', street_part, flags=re.IGNORECASE)
            street_part = re.sub(r'\b(street|st)\b', 'St', street_part, flags=re.IGNORECASE)
            street_part = re.sub(r'\b(road|rd)\b', 'Rd', street_part, flags=re.IGNORECASE)
            street_part = re.sub(r'\b(drive|dr)\b', 'Dr', street_part, flags=re.IGNORECASE)
            street_part = re.sub(r'\b(boulevard|blvd)\b', 'Blvd', street_part, flags=re.IGNORECASE)

            # 📬 Build USPS-like standardized address
            std_parts = [
                f"{number_part} {street_part}".strip(),
                f"{city_part}".strip(),
                f"{state_part} {zip_part}".strip()
            ]
            std_address = ", ".join([p for p in std_parts if p])
            std_address = re.sub(r'\s+', ' ', std_address)
            std_address = re.sub(r',\s*,', ',', std_address).strip(', ')

            df.loc[i, "StandardizedAddress"] = std_address
            df.loc[i, "AddressStatus"] = "Geocoded (fallback)"
        else:
            df.loc[i, "AddressStatus"] = "Not found (fallback failed)"

        # ⏳ Respect free-tier rate limits
        time.sleep(1)

# --- DROP UNUSED COLUMNS ---
drop_cols = [
    "CategoryCode", "Category", "Name", "Phone",
    "Status", "NotHomeAttempt", "NewFromPublisher",
    "Date1", "Date2", "Date3", "Date4", "Date5",
    "Language", "notes", "Notes", "NotesFromPublisher", "Process Notes"
]
df = df.drop(columns=drop_cols, errors="ignore")

# --- SAVE OUTPUT ---
output_path = f"output/standardized_output_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
df.to_csv(output_path, index=False)
print(f"✅ Processing complete! Results saved to {output_path}")


  0%|          | 0/19 [00:00<?, ?it/s]C:\Users\WINDOWS 10\AppData\Local\Temp\ipykernel_20100\2973812284.py:106: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '33.960921' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[i, "Latitude"] = lat
C:\Users\WINDOWS 10\AppData\Local\Temp\ipykernel_20100\2973812284.py:107: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-118.332084' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[i, "Longitude"] = lon
100%|██████████| 19/19 [00:22<00:00,  1.20s/it]

✅ Processing complete! Results saved to output/standardized_output_20251101_174115.csv


In [2]:
!pip install geopy rapidfuzz

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: C:\Python313\python.exe -m pip install --upgrade pip


In [7]:
!"{sys.executable}" -m pip install geopy rapidfuzz


  Using cached geopy-2.4.1-py3-none-any.whl.metadata (6.8 kB)
  Using cached geographiclib-2.1-py3-none-any.whl.metadata (1.6 kB)
Using cached geopy-2.4.1-py3-none-any.whl (125 kB)
Using cached geographiclib-2.1-py3-none-any.whl (40 kB)
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   --------------------------- ------------ 1.0/1.5 MB 10.1 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 4.8 MB/s eta 0:00:00

   ---------------------------------------- 0/3 [rapidfuzz]
   ---------------------------------------- 0/3 [rapidfuzz]
   ---------------------------------------- 0/3 [rapidfuzz]
   ---------------------------------------- 0/3 [rapidfuzz]
   ---------------------------------------- 0/3 [rapidfuzz]
   ------------- -------------------------- 1/3 [geographiclib]
   -------------------------- ------------- 2/3 [geopy]
   -------------------------- ------------- 2/3 [geopy]
   -------------------------- ------------- 2/3 [geopy]
   -----


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
from geopy.distance import geodesic
from rapidfuzz import fuzz
import pandas as pd
import time
from datetime import datetime

# Load your processed file (replace with the actual output file path)
df = pd.read_csv("output/standardized_output_20251101_174115.csv")

# --- Sanity check functions ---

def sanity_check_distance(orig_lat, orig_lon, new_lat, new_lon, threshold_km=0.5):
    """Checks if two coordinates are within threshold_km distance."""
    try:
        if not (pd.isna(orig_lat) or pd.isna(orig_lon) or pd.isna(new_lat) or pd.isna(new_lon)):
            dist = geodesic((float(orig_lat), float(orig_lon)), (float(new_lat), float(new_lon))).km
            return dist <= threshold_km, round(dist, 3)
    except:
        pass
    return False, None


def sanity_check_text(original_street, standardized_street, threshold=80):
    """Checks text similarity between original and standardized address."""
    if pd.notna(original_street) and pd.notna(standardized_street):
        similarity = fuzz.token_set_ratio(str(original_street).lower(), str(standardized_street).lower())
        return similarity >= threshold, similarity
    return False, 0


def sanity_check_region(orig_city, orig_zip, new_city, new_zip):
    """Checks if city/ZIP are still consistent."""
    try:
        if pd.notna(orig_zip) and pd.notna(new_zip):
            if str(orig_zip)[:3] != str(new_zip)[:3]:
                return False
        if pd.notna(orig_city) and pd.notna(new_city):
            if orig_city.lower()[:5] != new_city.lower()[:5]:
                return False
    except:
        return False
    return True


# --- Run sanity checks on each row ---
sanity_results = []

for i, row in df.iterrows():
    orig_addr = f"{row.get('Number', '')} {row.get('Street', '')}"
    std_addr = row.get("StandardizedAddress", "")
    orig_lat, orig_lon = row.get("Latitude", None), row.get("Longitude", None)
    new_lat, new_lon = row.get("Latitude", None), row.get("Longitude", None)  # can compare fallback if stored separately

    # Check text similarity
    ok_text, sim_score = sanity_check_text(orig_addr, std_addr)

    # Check distance (if comparing fallback vs original available)
    ok_dist, distance_km = sanity_check_distance(orig_lat, orig_lon, new_lat, new_lon)

    # Check region consistency
    ok_region = sanity_check_region(row.get("Suburb", ""), row.get("PostalCode", ""), row.get("Suburb", ""), row.get("PostalCode", ""))

    # Combine results
    sanity_status = "✅ OK"
    if not ok_text:
        sanity_status = f"⚠️ Low text match ({sim_score:.1f}%)"
    if not ok_region:
        sanity_status = f"⚠️ Region mismatch"
    if distance_km and distance_km > 0.5:
        sanity_status = f"⚠️ Far distance ({distance_km:.2f} km)"

    sanity_results.append(sanity_status)

df["SanityCheck"] = sanity_results

# --- Save sanity-checked output ---
output_path = "output/sanity_checked_" + datetime.now().strftime("%Y%m%d_%H%M%S") + ".csv"
df.to_csv(output_path, index=False)
print(f"✅ Sanity check complete! Results saved to {output_path}")

# --- Optional: Show flagged results ---
df[df["SanityCheck"].str.contains("⚠️", na=False)]


✅ Sanity check complete! Results saved to output/sanity_checked_20251101_175636.csv


,TerritoryID,TerritoryNumber,TerritoryAddressID,ApartmentNumber,Number,Street,Suburb,PostalCode,State,Type,Latitude,Longitude,StandardizedAddress,DPVStatus,AddressStatus,SanityCheck
2,786471,59,1622586,NaN,6916,10ave,Inglewood,NaN,NaN,House,33.975065,-118.369862,"6916 10ave, Inglewood, CA",NaN,Geocoded (fallback),⚠️ Low text match (57.1%)


Input sample 2

In [1]:
import os
import requests
import pandas as pd
from tqdm import tqdm
import time
from datetime import datetime
import re
import json

# --- CONFIGURATION ---
AUTH_ID = "c158cf93-8a4d-c43c-dade-f372cbeac850"
AUTH_TOKEN = "e7UMwEf0GevqQpxwdLaO"
SMARTY_URL = "https://us-street.api.smarty.com/street-address"

# Fallback Geocoder (used only when Smarty fails)
LOCATIONIQ_URL = "https://us1.locationiq.com/v1/search"
LOCATIONIQ_KEY = "pk.f25c0d747a60b0c3c2fc8632d2064efa"

# File paths
INPUT_FILE = "sample-input2.csv"
CACHE_DIR = "json_cache"
OUTPUT_DIR = "output"
os.makedirs(CACHE_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

# --- LOAD INPUT ---
df = pd.read_csv(INPUT_FILE)

# --- ADD OUTPUT COLUMNS ---
for col in ["StandardizedAddress", "DPVStatus", "Latitude", "Longitude", "AddressStatus", "IsDuplicate"]:
    if col not in df.columns:
        df[col] = ""

# --- HELPER FUNCTIONS ---
def cache_response(row_id, data):
    """Save full API response to JSON cache."""
    path = os.path.join(CACHE_DIR, f"{row_id}.json")
    with open(path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2)

def fallback_geocode(address):
    """LocationIQ fallback when Smarty fails."""
    try:
        params = {"key": LOCATIONIQ_KEY, "q": address, "format": "json", "limit": 1}
        r = requests.get(LOCATIONIQ_URL, params=params, timeout=10)
        data = r.json()
        if isinstance(data, list) and len(data) > 0:
            lat = float(data[0].get("lat", 0))
            lon = float(data[0].get("lon", 0))
            display_name = data[0].get("display_name", "")
            return lat, lon, display_name
    except Exception as e:
        print(f"⚠️ LocationIQ error for {address}: {e}")
    return None, None, None

def sanity_check(lat, lon, state):
    """Verify geocode is roughly within California bounds."""
    if not lat or not lon:
        return False
    try:
        lat, lon = float(lat), float(lon)
        return 32.0 <= lat <= 42.0 and -125.0 <= lon <= -114.0 and state.strip().upper() == "CA"
    except:
        return False

# --- MAIN PROCESSING LOOP ---
for i, row in tqdm(df.iterrows(), total=len(df)):
    number = str(row.get("Number", "")).strip()
    street = str(row.get("Street", "")).strip()
    city = str(row.get("Suburb", "")).strip()
    state = str(row.get("State", "CA")).strip()
    zipcode = str(row.get("PostalCode", "")).replace(".0", "").strip()

    # Skip empty or incomplete addresses
    if not street or not number:
        df.loc[i, "AddressStatus"] = "Missing street/number"
        continue

    # --- Call Smarty API ---
    params = {
        "auth-id": AUTH_ID,
        "auth-token": AUTH_TOKEN,
        "street": f"{number} {street}",
        "city": city,
        "state": state,
        "zipcode": zipcode,
    }

    try:
        response = requests.get(SMARTY_URL, params=params, timeout=10)
        data = response.json()
        cache_response(i, data)

        if response.status_code == 200 and data:
            result = data[0]
            components = result.get("components", {})
            metadata = result.get("metadata", {})
            analysis = result.get("analysis", {})

            # ✅ Construct USPS-style standardized address
            line1 = result.get("delivery_line_1", "")
            line2 = result.get("delivery_line_2", "")
            last_line = result.get("last_line", "")
            standardized = f"{line1} {line2}, {last_line}".strip().replace(" ,", ",")

            df.loc[i, "StandardizedAddress"] = standardized
            df.loc[i, "DPVStatus"] = analysis.get("dpv_match_code", "")
            df.loc[i, "Latitude"] = metadata.get("latitude", "")
            df.loc[i, "Longitude"] = metadata.get("longitude", "")
            df.loc[i, "AddressStatus"] = "Validated"
        else:
            df.loc[i, "AddressStatus"] = "No match (Smarty failed)"

    except Exception as e:
        print(f"⚠️ Smarty API error for {street}: {e}")
        df.loc[i, "AddressStatus"] = "Error"

    # --- Fallback for missing or invalid geocode ---
    lat_val = df.loc[i, "Latitude"]
    lon_val = df.loc[i, "Longitude"]

    if not sanity_check(lat_val, lon_val, state):
        full_addr = f"{number} {street}, {city}, {state} {zipcode}"
        lat, lon, display_name = fallback_geocode(full_addr)

        if lat and lon and sanity_check(lat, lon, state):
            df.loc[i, "Latitude"] = lat
            df.loc[i, "Longitude"] = lon
            df.loc[i, "StandardizedAddress"] = f"{number} {street.title()}, {city.title()}, {state.upper()} {zipcode}"
            df.loc[i, "AddressStatus"] = "Geocoded (fallback)"
        else:
            df.loc[i, "AddressStatus"] = "Not found (fallback failed)"

        time.sleep(1)

# --- QUALITY CHECKS ---
df["IsDuplicate"] = df.duplicated(subset=["StandardizedAddress"], keep=False)

def classify_address_issue(row):
    if row["DPVStatus"] == "N":
        return "Invalid or Undeliverable"
    elif not row["StandardizedAddress"]:
        return "Could not standardize"
    elif row["IsDuplicate"]:
        return "Duplicate"
    elif "CA" not in row["StandardizedAddress"]:
        return "Sanity check fail (out of CA)"
    else:
        return "Clean"

df["AddressStatus"] = df.apply(classify_address_issue, axis=1)

# --- QUALITY SUMMARY ---
print("\n📊 Address Quality Summary:")
print(df["AddressStatus"].value_counts())

# --- SAVE OUTPUT ---
output_path = os.path.join(
    OUTPUT_DIR,
    f"addresses_standardized_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
)
df.to_csv(output_path, index=False)
print(f"\n✅ Processing complete! Results saved to {output_path}")


100%|██████████| 10/10 [00:06<00:00,  1.49it/s]


📊 Address Quality Summary:
AddressStatus
Clean    10
Name: count, dtype: int64

✅ Processing complete! Results saved to output\addresses_standardized_20251101_184443.csv


Input 3

In [2]:
import os
import requests
import pandas as pd
from tqdm import tqdm
import time
from datetime import datetime
import re
import json

# --- CONFIGURATION ---
AUTH_ID = "c158cf93-8a4d-c43c-dade-f372cbeac850"
AUTH_TOKEN = "e7UMwEf0GevqQpxwdLaO"
SMARTY_URL = "https://us-street.api.smarty.com/street-address"

# Fallback Geocoder (used only when Smarty fails)
LOCATIONIQ_URL = "https://us1.locationiq.com/v1/search"
LOCATIONIQ_KEY = "pk.f25c0d747a60b0c3c2fc8632d2064efa"

# File paths
INPUT_FILE = "sample-input3.csv"
CACHE_DIR = "json_cache"
OUTPUT_DIR = "output"
os.makedirs(CACHE_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

# --- LOAD INPUT ---
df = pd.read_csv(INPUT_FILE)

# --- ADD OUTPUT COLUMNS ---
for col in ["StandardizedAddress", "DPVStatus", "Latitude", "Longitude", "AddressStatus", "IsDuplicate"]:
    if col not in df.columns:
        df[col] = ""

# --- HELPER FUNCTIONS ---
def cache_response(row_id, data):
    """Save full API response to JSON cache."""
    path = os.path.join(CACHE_DIR, f"{row_id}.json")
    with open(path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2)

def fallback_geocode(address):
    """LocationIQ fallback when Smarty fails."""
    try:
        params = {"key": LOCATIONIQ_KEY, "q": address, "format": "json", "limit": 1}
        r = requests.get(LOCATIONIQ_URL, params=params, timeout=10)
        data = r.json()
        if isinstance(data, list) and len(data) > 0:
            lat = float(data[0].get("lat", 0))
            lon = float(data[0].get("lon", 0))
            display_name = data[0].get("display_name", "")
            return lat, lon, display_name
    except Exception as e:
        print(f"⚠️ LocationIQ error for {address}: {e}")
    return None, None, None

def sanity_check(lat, lon, state):
    """Verify geocode is roughly within California bounds."""
    if not lat or not lon:
        return False
    try:
        lat, lon = float(lat), float(lon)
        return 32.0 <= lat <= 42.0 and -125.0 <= lon <= -114.0 and state.strip().upper() == "CA"
    except:
        return False

# --- MAIN PROCESSING LOOP ---
for i, row in tqdm(df.iterrows(), total=len(df)):
    number = str(row.get("Number", "")).strip()
    street = str(row.get("Street", "")).strip()
    city = str(row.get("Suburb", "")).strip()
    state = str(row.get("State", "CA")).strip()
    zipcode = str(row.get("PostalCode", "")).replace(".0", "").strip()

    # Skip empty or incomplete addresses
    if not street or not number:
        df.loc[i, "AddressStatus"] = "Missing street/number"
        continue

    # --- Call Smarty API ---
    params = {
        "auth-id": AUTH_ID,
        "auth-token": AUTH_TOKEN,
        "street": f"{number} {street}",
        "city": city,
        "state": state,
        "zipcode": zipcode,
    }

    try:
        response = requests.get(SMARTY_URL, params=params, timeout=10)
        data = response.json()
        cache_response(i, data)

        if response.status_code == 200 and data:
            result = data[0]
            components = result.get("components", {})
            metadata = result.get("metadata", {})
            analysis = result.get("analysis", {})

            # ✅ Construct USPS-style standardized address
            line1 = result.get("delivery_line_1", "")
            line2 = result.get("delivery_line_2", "")
            last_line = result.get("last_line", "")
            standardized = f"{line1} {line2}, {last_line}".strip().replace(" ,", ",")

            df.loc[i, "StandardizedAddress"] = standardized
            df.loc[i, "DPVStatus"] = analysis.get("dpv_match_code", "")
            df.loc[i, "Latitude"] = metadata.get("latitude", "")
            df.loc[i, "Longitude"] = metadata.get("longitude", "")
            df.loc[i, "AddressStatus"] = "Validated"

            # --- CONFIDENCE SCORING ---
            def compute_confidence(row):
                if row["AddressStatus"] == "Clean" and row["DPVStatus"] == "Y":
                    return "High"
                elif row["AddressStatus"] in ["Clean", "Geocoded (fallback)"] and row["DPVStatus"] in ["S", "D", "Y"]:
                    return "Medium"
                elif "Sanity" in str(row["AddressStatus"]) or "fallback" in str(row["AddressStatus"]):
                    return "Low"
                else:
                    return "Unknown"

            df["ConfidenceLevel"] = df.apply(compute_confidence, axis=1)


        else:
            df.loc[i, "AddressStatus"] = "No match (Smarty failed)"

    except Exception as e:
        print(f"⚠️ Smarty API error for {street}: {e}")
        df.loc[i, "AddressStatus"] = "Error"

    # --- Fallback for missing or invalid geocode ---
    lat_val = df.loc[i, "Latitude"]
    lon_val = df.loc[i, "Longitude"]

    if not sanity_check(lat_val, lon_val, state):
        full_addr = f"{number} {street}, {city}, {state} {zipcode}"
        lat, lon, display_name = fallback_geocode(full_addr)

        if lat and lon and sanity_check(lat, lon, state):
            df.loc[i, "Latitude"] = lat
            df.loc[i, "Longitude"] = lon
            df.loc[i, "StandardizedAddress"] = f"{number} {street.title()}, {city.title()}, {state.upper()} {zipcode}"
            df.loc[i, "AddressStatus"] = "Geocoded (fallback)"
        else:
            df.loc[i, "AddressStatus"] = "Not found (fallback failed)"

        time.sleep(1)

# --- QUALITY CHECKS ---
df["IsDuplicate"] = df.duplicated(subset=["StandardizedAddress"], keep=False)

def classify_address_issue(row):
    if row["DPVStatus"] == "N":
        return "Invalid or Undeliverable"
    elif not row["StandardizedAddress"]:
        return "Could not standardize"
    elif row["IsDuplicate"]:
        return "Duplicate"
    elif "CA" not in row["StandardizedAddress"]:
        return "Sanity check fail (out of CA)"
    else:
        return "Clean"

df["AddressStatus"] = df.apply(classify_address_issue, axis=1)

# --- QUALITY SUMMARY ---
print("\n📊 Address Quality Summary:")
print(df["AddressStatus"].value_counts())

# --- SAVE OUTPUT ---
output_path = os.path.join(
    OUTPUT_DIR,
    f"addresses_standardized_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
)
df.to_csv(output_path, index=False)
print(f"\n✅ Processing complete! Results saved to {output_path}")


100%|██████████| 10/10 [00:13<00:00,  1.33s/it]


📊 Address Quality Summary:
AddressStatus
Clean                    6
Could not standardize    4
Name: count, dtype: int64

✅ Processing complete! Results saved to output\addresses_standardized_20251101_185537.csv


In [3]:
import os
import requests
import pandas as pd
from tqdm import tqdm
import time
from datetime import datetime
import re
import json
import math


# --- CONFIGURATION ---
AUTH_ID = "c158cf93-8a4d-c43c-dade-f372cbeac850"
AUTH_TOKEN = "e7UMwEf0GevqQpxwdLaO"
SMARTY_URL = "https://us-street.api.smarty.com/street-address"

# Fallback Geocoder (used only when Smarty fails)
LOCATIONIQ_URL = "https://us1.locationiq.com/v1/search"
LOCATIONIQ_KEY = "pk.f25c0d747a60b0c3c2fc8632d2064efa"

# File paths
INPUT_FILE = "sample-input3.csv"
CACHE_DIR = "json_cache"
OUTPUT_DIR = "output"
os.makedirs(CACHE_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

# --- LOAD INPUT ---
df = pd.read_csv(INPUT_FILE)

# --- ADD OUTPUT COLUMNS ---
for col in ["StandardizedAddress", "DPVStatus", "Latitude", "Longitude", "AddressStatus", "IsDuplicate"]:
    if col not in df.columns:
        df[col] = ""

# --- HELPER FUNCTIONS ---
def cache_response(row_id, data):
    """Save full API response to JSON cache."""
    path = os.path.join(CACHE_DIR, f"{row_id}.json")
    with open(path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2)

def fallback_geocode(address):
    """LocationIQ fallback when Smarty fails."""
    try:
        params = {"key": LOCATIONIQ_KEY, "q": address, "format": "json", "limit": 1}
        r = requests.get(LOCATIONIQ_URL, params=params, timeout=10)
        data = r.json()
        if isinstance(data, list) and len(data) > 0:
            lat = float(data[0].get("lat", 0))
            lon = float(data[0].get("lon", 0))
            display_name = data[0].get("display_name", "")
            return lat, lon, display_name
    except Exception as e:
        print(f"⚠️ LocationIQ error for {address}: {e}")
    return None, None, None

def sanity_check(lat, lon, state):
    """Verify geocode is roughly within California bounds."""
    if not lat or not lon:
        return False
    try:
        lat, lon = float(lat), float(lon)
        return 32.0 <= lat <= 42.0 and -125.0 <= lon <= -114.0 and state.strip().upper() == "CA"
    except:
        return False

# --- MAIN PROCESSING LOOP ---
for i, row in tqdm(df.iterrows(), total=len(df)):
    number = str(row.get("Number", "")).strip()
    street = str(row.get("Street", "")).strip()
    city = str(row.get("Suburb", "")).strip()
    state = str(row.get("State", "CA")).strip()
    zipcode = str(row.get("PostalCode", "")).replace(".0", "").strip()

    # Skip empty or incomplete addresses
    if not street or not number:
        df.loc[i, "AddressStatus"] = "Missing street/number"
        continue

    # --- Call Smarty API ---
    params = {
        "auth-id": AUTH_ID,
        "auth-token": AUTH_TOKEN,
        "street": f"{number} {street}",
        "city": city,
        "state": state,
        "zipcode": zipcode,
    }

    try:
        response = requests.get(SMARTY_URL, params=params, timeout=10)
        data = response.json()
        cache_response(i, data)

        if response.status_code == 200 and data:
            result = data[0]
            components = result.get("components", {})
            metadata = result.get("metadata", {})
            analysis = result.get("analysis", {})

            # ✅ Construct USPS-style standardized address
            line1 = result.get("delivery_line_1", "")
            line2 = result.get("delivery_line_2", "")
            last_line = result.get("last_line", "")
            standardized = f"{line1} {line2}, {last_line}".strip().replace(" ,", ",")

            df.loc[i, "StandardizedAddress"] = standardized
            df.loc[i, "DPVStatus"] = analysis.get("dpv_match_code", "")
            df.loc[i, "Latitude"] = metadata.get("latitude", "")
            df.loc[i, "Longitude"] = metadata.get("longitude", "")
            df.loc[i, "AddressStatus"] = "Validated"

            # --- CONFIDENCE: categorical + numeric score ---


            def compute_confidence_pair(row):
                """
                Return (label, score) where score is 0..1 (higher = more confident).
                Rules (explicit and conservative):
                - Clean & DPV Y        -> High (0.95)
                - Clean & DPV S or D   -> Medium (0.75)
                - Clean & DPV blank    -> Medium (0.65)  # Smarty standardized but DPV missing; still OK
                - Geocoded (fallback) & sanity OK -> Medium (0.70)
                - Geocoded (fallback) & sanity FAIL -> Low (0.30)
                - Duplicate            -> Low (0.35) if not DPV Y, else Medium-Low (0.5)
                - Invalid/Undeliverable (DPV N) -> Low (0.15)
                - Could not standardize -> Unknown/Low (0.20)
                - Any other unexpected -> Unknown (0.50)
                """
                status = str(row.get("AddressStatus", "")).strip()
                dpv = str(row.get("DPVStatus", "")).upper().strip()
                is_dup = bool(row.get("IsDuplicate", False))
                # optional: sanity flag present (if you computed it earlier)
                sanity = str(row.get("SanityFlag", "")).lower()

                # Clean rows
                if status == "Clean":
                    if dpv == "Y":
                        return "High", 0.95
                    if dpv in ("S", "D"):
                        return "Medium", 0.75
                    if dpv in ("", "NONE", "NAN"):
                        # standardized but DPV not present — treat as medium confidence
                        return "Medium", 0.65
                    # fallback
                    return "Medium", 0.6

                # Smarty validated but flagged by sanity (e.g., out-of-state)
                if "sanity" in status.lower() or "out of" in status.lower():
                    return "Low", 0.25

                # Fallback geocode
                if "geocoded" in status.lower() or "fallback" in status.lower():
                    if sanity == "ok" or "ok" in str(row.get("SanityFlag", "")).lower():
                        return "Medium", 0.70
                    return "Low", 0.30

                # Duplicate rows
                if status == "Duplicate" or is_dup:
                    # if duplicates but DPV Y -> still medium-low confidence
                    if dpv == "Y":
                        return "Medium-Low", 0.50
                    return "Low", 0.35

                # Invalid/Undeliverable
                if status == "Invalid or Undeliverable" or dpv == "N":
                    return "Low", 0.15

                # Could not standardize
                if status == "Could not standardize":
                    return "Unknown", 0.20

                # Default fallback
                return "Unknown", 0.50

            # apply and split into two columns
            pairs = df.apply(compute_confidence_pair, axis=1)
            df["ConfidenceLevel"] = pairs.apply(lambda p: p[0])
            df["ConfidenceScore"] = pairs.apply(lambda p: p[1])

            # Optional: make ConfidenceScore numeric and fill missing
            df["ConfidenceScore"] = pd.to_numeric(df["ConfidenceScore"], errors="coerce").fillna(0.5)



        else:
            df.loc[i, "AddressStatus"] = "No match (Smarty failed)"

    except Exception as e:
        print(f"⚠️ Smarty API error for {street}: {e}")
        df.loc[i, "AddressStatus"] = "Error"

    # --- Fallback for missing or invalid geocode ---
    lat_val = df.loc[i, "Latitude"]
    lon_val = df.loc[i, "Longitude"]

    if not sanity_check(lat_val, lon_val, state):
        full_addr = f"{number} {street}, {city}, {state} {zipcode}"
        lat, lon, display_name = fallback_geocode(full_addr)

        if lat and lon and sanity_check(lat, lon, state):
            df.loc[i, "Latitude"] = lat
            df.loc[i, "Longitude"] = lon
            df.loc[i, "StandardizedAddress"] = f"{number} {street.title()}, {city.title()}, {state.upper()} {zipcode}"
            df.loc[i, "AddressStatus"] = "Geocoded (fallback)"
        else:
            df.loc[i, "AddressStatus"] = "Not found (fallback failed)"

        time.sleep(1)

# --- QUALITY CHECKS ---
df["IsDuplicate"] = df.duplicated(subset=["StandardizedAddress"], keep=False)

def classify_address_issue(row):
    if row["DPVStatus"] == "N":
        return "Invalid or Undeliverable"
    elif not row["StandardizedAddress"]:
        return "Could not standardize"
    elif row["IsDuplicate"]:
        return "Duplicate"
    elif "CA" not in row["StandardizedAddress"]:
        return "Sanity check fail (out of CA)"
    else:
        return "Clean"

df["AddressStatus"] = df.apply(classify_address_issue, axis=1)

# --- QUALITY SUMMARY ---
print("\n📊 Address Quality Summary:")
print(df["AddressStatus"].value_counts())

# --- SAVE OUTPUT ---
output_path = os.path.join(
    OUTPUT_DIR,
    f"addresses_standardized_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
)
df.to_csv(output_path, index=False)
print(f"\n✅ Processing complete! Results saved to {output_path}")


100%|██████████| 10/10 [00:13<00:00,  1.34s/it]


📊 Address Quality Summary:
AddressStatus
Clean                    6
Could not standardize    4
Name: count, dtype: int64

✅ Processing complete! Results saved to output\addresses_standardized_20251101_190653.csv


In [4]:
import os
import requests
import pandas as pd
from tqdm import tqdm
import time
from datetime import datetime
import re
import json
import math


# --- CONFIGURATION ---
AUTH_ID = "c158cf93-8a4d-c43c-dade-f372cbeac850"
AUTH_TOKEN = "e7UMwEf0GevqQpxwdLaO"
SMARTY_URL = "https://us-street.api.smarty.com/street-address"

# Fallback Geocoder (used only when Smarty fails)
LOCATIONIQ_URL = "https://us1.locationiq.com/v1/search"
LOCATIONIQ_KEY = "pk.f25c0d747a60b0c3c2fc8632d2064efa"

# File paths
INPUT_FILE = "sample-input3.csv"
CACHE_DIR = "json_cache"
OUTPUT_DIR = "output"
os.makedirs(CACHE_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

# --- LOAD INPUT ---
df = pd.read_csv(INPUT_FILE)

# --- ADD OUTPUT COLUMNS ---
for col in ["StandardizedAddress", "DPVStatus", "Latitude", "Longitude", "AddressStatus", "IsDuplicate"]:
    if col not in df.columns:
        df[col] = ""

# --- HELPER FUNCTIONS ---
def cache_response(row_id, data):
    """Save full API response to JSON cache."""
    path = os.path.join(CACHE_DIR, f"{row_id}.json")
    with open(path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2)

def fallback_geocode(address):
    """LocationIQ fallback when Smarty fails."""
    try:
        params = {"key": LOCATIONIQ_KEY, "q": address, "format": "json", "limit": 1}
        r = requests.get(LOCATIONIQ_URL, params=params, timeout=10)
        data = r.json()
        if isinstance(data, list) and len(data) > 0:
            lat = float(data[0].get("lat", 0))
            lon = float(data[0].get("lon", 0))
            display_name = data[0].get("display_name", "")
            return lat, lon, display_name
    except Exception as e:
        print(f"⚠️ LocationIQ error for {address}: {e}")
    return None, None, None

def sanity_check(lat, lon, state):
    """Verify geocode is roughly within California bounds."""
    if not lat or not lon:
        return False
    try:
        lat, lon = float(lat), float(lon)
        return 32.0 <= lat <= 42.0 and -125.0 <= lon <= -114.0 and state.strip().upper() == "CA"
    except:
        return False

# --- MAIN PROCESSING LOOP ---
for i, row in tqdm(df.iterrows(), total=len(df)):
    number = str(row.get("Number", "")).strip()
    street = str(row.get("Street", "")).strip()
    city = str(row.get("Suburb", "")).strip()
    state = str(row.get("State", "CA")).strip()
    zipcode = str(row.get("PostalCode", "")).replace(".0", "").strip()

    # Skip empty or incomplete addresses
    if not street or not number:
        df.loc[i, "AddressStatus"] = "Missing street/number"
        continue

    # --- Call Smarty API ---
    params = {
        "auth-id": AUTH_ID,
        "auth-token": AUTH_TOKEN,
        "street": f"{number} {street}",
        "city": city,
        "state": state,
        "zipcode": zipcode,
    }

    try:
        response = requests.get(SMARTY_URL, params=params, timeout=10)
        data = response.json()
        cache_response(i, data)

        if response.status_code == 200 and data:
            result = data[0]
            components = result.get("components", {})
            metadata = result.get("metadata", {})
            analysis = result.get("analysis", {})

            # ✅ Construct USPS-style standardized address
            line1 = result.get("delivery_line_1", "")
            line2 = result.get("delivery_line_2", "")
            last_line = result.get("last_line", "")
            standardized = f"{line1} {line2}, {last_line}".strip().replace(" ,", ",")

            df.loc[i, "StandardizedAddress"] = standardized
            df.loc[i, "DPVStatus"] = analysis.get("dpv_match_code", "")
            df.loc[i, "Latitude"] = metadata.get("latitude", "")
            df.loc[i, "Longitude"] = metadata.get("longitude", "")
            df.loc[i, "AddressStatus"] = "Validated"
            # --- 🧱 Fill missing address parts using Smarty components ---
            if pd.isna(row.get("PostalCode")) or not str(row.get("PostalCode")).strip():
                df.loc[i, "PostalCode"] = components.get("zipcode", "")
            if pd.isna(row.get("State")) or not str(row.get("State")).strip():
                df.loc[i, "State"] = components.get("state_abbreviation", "")
            if pd.isna(row.get("Suburb")) or not str(row.get("Suburb")).strip():
                df.loc[i, "Suburb"] = components.get("city_name", "")
            if pd.isna(row.get("Street")) or not str(row.get("Street")).strip():
                street_parts = [
                    components.get("street_predirection", ""),
                    components.get("street_name", ""),
                    components.get("street_suffix", "")
                ]
                df.loc[i, "Street"] = " ".join([p for p in street_parts if p]).strip()
            if pd.isna(row.get("Number")) or not str(row.get("Number")).strip():
                df.loc[i, "Number"] = components.get("primary_number", "")

            # --- CONFIDENCE: categorical + numeric score ---


            def compute_confidence_pair(row):
                """
                Return (label, score) where score is 0..1 (higher = more confident).
                Rules (explicit and conservative):
                - Clean & DPV Y        -> High (0.95)
                - Clean & DPV S or D   -> Medium (0.75)
                - Clean & DPV blank    -> Medium (0.65)  # Smarty standardized but DPV missing; still OK
                - Geocoded (fallback) & sanity OK -> Medium (0.70)
                - Geocoded (fallback) & sanity FAIL -> Low (0.30)
                - Duplicate            -> Low (0.35) if not DPV Y, else Medium-Low (0.5)
                - Invalid/Undeliverable (DPV N) -> Low (0.15)
                - Could not standardize -> Unknown/Low (0.20)
                - Any other unexpected -> Unknown (0.50)
                """
                status = str(row.get("AddressStatus", "")).strip()
                dpv = str(row.get("DPVStatus", "")).upper().strip()
                is_dup = bool(row.get("IsDuplicate", False))
                # optional: sanity flag present (if you computed it earlier)
                sanity = str(row.get("SanityFlag", "")).lower()

                # Clean rows
                if status == "Clean":
                    if dpv == "Y":
                        return "High", 0.95
                    if dpv in ("S", "D"):
                        return "Medium", 0.75
                    if dpv in ("", "NONE", "NAN"):
                        # standardized but DPV not present — treat as medium confidence
                        return "Medium", 0.65
                    # fallback
                    return "Medium", 0.6

                # Smarty validated but flagged by sanity (e.g., out-of-state)
                if "sanity" in status.lower() or "out of" in status.lower():
                    return "Low", 0.25

                # Fallback geocode
                if "geocoded" in status.lower() or "fallback" in status.lower():
                    if sanity == "ok" or "ok" in str(row.get("SanityFlag", "")).lower():
                        return "Medium", 0.70
                    return "Low", 0.30

                # Duplicate rows
                if status == "Duplicate" or is_dup:
                    # if duplicates but DPV Y -> still medium-low confidence
                    if dpv == "Y":
                        return "Medium-Low", 0.50
                    return "Low", 0.35

                # Invalid/Undeliverable
                if status == "Invalid or Undeliverable" or dpv == "N":
                    return "Low", 0.15

                # Could not standardize
                if status == "Could not standardize":
                    return "Unknown", 0.20

                # Default fallback
                return "Unknown", 0.50

            # apply and split into two columns
            pairs = df.apply(compute_confidence_pair, axis=1)
            df["ConfidenceLevel"] = pairs.apply(lambda p: p[0])
            df["ConfidenceScore"] = pairs.apply(lambda p: p[1])

            # Optional: make ConfidenceScore numeric and fill missing
            df["ConfidenceScore"] = pd.to_numeric(df["ConfidenceScore"], errors="coerce").fillna(0.5)



        else:
            df.loc[i, "AddressStatus"] = "No match (Smarty failed)"

    except Exception as e:
        print(f"⚠️ Smarty API error for {street}: {e}")
        df.loc[i, "AddressStatus"] = "Error"

    # --- Fallback for missing or invalid geocode ---
    lat_val = df.loc[i, "Latitude"]
    lon_val = df.loc[i, "Longitude"]

    if not sanity_check(lat_val, lon_val, state):
        full_addr = f"{number} {street}, {city}, {state} {zipcode}"
        lat, lon, display_name = fallback_geocode(full_addr)

        if lat and lon and sanity_check(lat, lon, state):
            df.loc[i, "Latitude"] = lat
            df.loc[i, "Longitude"] = lon
            df.loc[i, "StandardizedAddress"] = f"{number} {street.title()}, {city.title()}, {state.upper()} {zipcode}"
            df.loc[i, "AddressStatus"] = "Geocoded (fallback)"
        else:
            df.loc[i, "AddressStatus"] = "Not found (fallback failed)"

        time.sleep(1)

# --- QUALITY CHECKS ---
df["IsDuplicate"] = df.duplicated(subset=["StandardizedAddress"], keep=False)

def classify_address_issue(row):
    if row["DPVStatus"] == "N":
        return "Invalid or Undeliverable"
    elif not row["StandardizedAddress"]:
        return "Could not standardize"
    elif row["IsDuplicate"]:
        return "Duplicate"
    elif "CA" not in row["StandardizedAddress"]:
        return "Sanity check fail (out of CA)"
    else:
        return "Clean"

df["AddressStatus"] = df.apply(classify_address_issue, axis=1)

# --- QUALITY SUMMARY ---
print("\n📊 Address Quality Summary:")
print(df["AddressStatus"].value_counts())

# --- SAVE OUTPUT ---
output_path = os.path.join(
    OUTPUT_DIR,
    f"addresses_standardized_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
)
df.to_csv(output_path, index=False)
print(f"\n✅ Processing complete! Results saved to {output_path}")


100%|██████████| 10/10 [00:13<00:00,  1.37s/it]


📊 Address Quality Summary:
AddressStatus
Clean                    6
Could not standardize    4
Name: count, dtype: int64

✅ Processing complete! Results saved to output\addresses_standardized_20251101_192026.csv
